In [1]:
import folium

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from folium.plugins import MarkerCluster


In [38]:
m = folium.Map(location=[51.5, -0.1], zoom_start=15)

In [39]:
m

In [62]:
accident_data = pd.read_csv("dataWith5000RNRecordsWithIDAndHeaders")
accident_data = accident_data.sample(n=5000, random_state=42)
accident_data.dropna(subset=["Latitude", "Longitude"], inplace=True)

In [63]:
accident_data.shape

(5000, 32)

In [68]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [129]:
 def generate_heat_gridmap(accident_data, htmlfilename):
    m = folium.Map(zoom_start = 5, location=[55, 0])

    # Generate GeoJson grid
    top_right = [58, 2]
    top_left = [49, -8]

    grid = get_geojson_grid(top_right, top_left, n=6)

    # Calculate exposures in grid
    popups = []
    regional_counts = []

    for box in grid:
        upper_right = box["properties"]["upper_right"]
        lower_left = box["properties"]["lower_left"]

        mask = (
            (accident_data.Latitude != upper_right[1]) & (accident_data.Latitude != lower_left[1]) &
            (accident_data.Longitude != upper_right[0]) & (accident_data.Longitude != lower_left[0])
               )

        region_incidents = len(accident_data[mask])
        regional_counts.append(region_incidents)

        total_vehicles = accident_data[mask].Number_of_Vehicles.sum()
        total_casualties = accident_data[mask].Number_of_Casualties.sum()
        content = "total vehicles {:,.0f}, total casualties {:,.0f}".format(total_vehicles, total_casualties)
        popup = folium.Popup(content)
        popups.append(popup)

    worst_region = max(regional_counts)

    # Add GeoJson to map
    for i, box in enumerate(grid):
        geo_json = json.dumps(box)

        color = plt.cm.Reds(regional_counts[i] / worst_region)
        color = mpl.colors.to_hex(color)

        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature, color=color: {
                                                                            'fillColor': color,
                                                                            'color':"black",
                                                                            'weight': 2,
                                                                            'dashArray': '5, 5',
                                                                            'fillOpacity': 0.55,
                                                                        })

        gj.add_child(popups[i])
        m.add_child(gj)

    # Marker clusters
    locations = list(zip(accident_data.Latitude, accident_data.Longitude))
    icons = [folium.Icon(icon="car", prefix="fa") for _ in range(len(locations))]

    # Create popups
    popup_content = []
    for incident in accident_data.itertuples():
        number_of_vehicles = "Number of vehicles: {} ".format(incident.Number_of_Vehicles)
        number_of_casualties = "Number of casualties: {}".format(incident.Number_of_Casualties)
        content = number_of_vehicles + number_of_casualties
        popup_content.append(content)

    popups = [folium.Popup(content) for content in popup_content]

    cluster = MarkerCluster(locations=locations, icons=icons, popups=popups)
    m.add_child(cluster)

    m.save(htmlfilename)

In [180]:
generate_heat_gridmap(accident_data, "car_accidents_all.html")

Uptill here, we saw the visualization for the whole dataset
Now lets see how the data is spread based on various light and weather conditions

In [73]:
accident_data_whole = pd.read_csv("dataWith5000RNRecordsWithIDAndHeaders")

In [102]:
sorted(accident_data_whole['Light_Conditions'].unique())

[1, 4, 5, 6, 7]

In [101]:
sorted(accident_data_whole['Weather_Conditions'].unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9]

**Assigning separate dataframes with records based on separate light conditions

All we got to do now is, call the generate_heat_gridmap to plot the heatmap grid for various datasets below

Generating gridmap for Light Conditions == 1

In [131]:
#below one has a shape of (3699,32)
accident_light1_df = accident_data_whole[accident_data_whole['Light_Conditions'] == 1]

In [133]:
accident_light1_df.shape

(3699, 32)

In [132]:
generate_heat_gridmap(accident_light1_df, "accident_light1.html")

Generating gridmap for Light Conditions == 4

In [134]:
#below one has a shape of (995,32)
accident_light4_df = accident_data_whole[accident_data_whole['Light_Conditions'] == 4]

In [135]:
accident_light4_df.shape

(995, 32)

In [136]:
generate_heat_gridmap(accident_light4_df, "accident_light4.html")

Generating gridmap for Light Conditions == 5

In [137]:
#below one has a shape of (33,32)
accident_light5_df = accident_data_whole[accident_data_whole['Light_Conditions'] == 5]

In [138]:
accident_light5_df.shape

(33, 32)

In [139]:
generate_heat_gridmap(accident_light5_df, "accident_light5.html")

Generating gridmap for Light Conditions == 6

In [140]:
#below one has a shape of (216,32)
accident_light6_df = accident_data_whole[accident_data_whole['Light_Conditions'] == 6]

In [141]:
accident_light6_df.shape

(216, 32)

In [142]:
generate_heat_gridmap(accident_light6_df, "accident_light6.html")

Generating gridmap for Light Conditions == 7

In [143]:
#below one has a shape of (57,32)
accident_light7_df = accident_data_whole[accident_data_whole['Light_Conditions'] == 7]

In [144]:
accident_light7_df.shape

(57, 32)

In [145]:
generate_heat_gridmap(accident_light7_df, "accident_light7.html")

**Assigning separate dataframes with records based on same light and weather conditions

In [146]:
#below one has a shape of (3159,32)
accident_light_weather = accident_data_whole[accident_data_whole['Light_Conditions'] == accident_data_whole['Weather_Conditions']]

In [147]:
accident_light_weather.shape

(3159, 32)

In [148]:
generate_heat_gridmap(accident_light_weather, "accident_light_weather.html")

**Assigning separate dataframes with records based on separate weather conditions
unique weather:[1, 2, 3, 4, 5, 6, 7, 8, 9]

Generating gridmap for Weather Conditions == 1

In [151]:
#below one has a shape of (4065, 32)
accident_weather1_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 1]

In [152]:
accident_weather1_df.shape

(4065, 32)

In [170]:
generate_heat_gridmap(accident_weather1_df, "accident_weather1.html")

Generating gridmap for Weather Conditions == 2

In [154]:
#below one has a shape of (585, 32)
accident_weather2_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 2]

In [155]:
accident_weather2_df.shape

(585, 32)

In [171]:
generate_heat_gridmap(accident_weather2_df, "accident_weather2.html")

Generating gridmap for Weather Conditions == 3

In [156]:
#below one has a shape of (10, 32)
accident_weather3_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 3]

In [157]:
accident_weather3_df.shape

(10, 32)

In [172]:
generate_heat_gridmap(accident_weather3_df, "accident_weather3.html")

Generating gridmap for Weather Conditions == 4

In [158]:
#below one has a shape of (47, 32)
accident_weather4_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 4]

In [159]:
accident_weather4_df.shape

(47, 32)

In [173]:
generate_heat_gridmap(accident_weather4_df, "accident_weather4.html")

Generating gridmap for Weather Conditions == 5

In [160]:
#below one has a shape of (88, 32)
accident_weather5_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 5]

In [161]:
accident_weather5_df.shape

(88, 32)

In [174]:
generate_heat_gridmap(accident_weather5_df, "accident_weather5.html")

Generating gridmap for Weather Conditions == 6

In [162]:
#below one has a shape of (1, 32)
accident_weather6_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 6]

In [163]:
accident_weather6_df.shape

(1, 32)

In [175]:
generate_heat_gridmap(accident_weather6_df, "accident_weather6.html")

Generating gridmap for Weather Conditions == 7

In [164]:
#below one has a shape of (22, 32)
accident_weather7_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 7]

In [165]:
accident_weather7_df.shape

(22, 32)

In [176]:
generate_heat_gridmap(accident_weather7_df, "accident_weather7.html")

Generating gridmap for Weather Conditions == 8

In [166]:
#below one has a shape of (92, 32)
accident_weather8_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 8]

In [167]:
accident_weather8_df.shape

(92, 32)

In [177]:
generate_heat_gridmap(accident_weather8_df, "accident_weather8.html")

Generating gridmap for Weather Conditions == 9

In [168]:
#below one has a shape of (90, 32)
accident_weather9_df = accident_data_whole[accident_data_whole['Weather_Conditions'] == 9]

In [169]:
accident_weather9_df.shape

(90, 32)

In [178]:
generate_heat_gridmap(accident_weather9_df, "accident_weather9.html")

Rough Work below

accident_data_whole[accident_data_whole['Light_Conditions'] == 7][['Latitude', 'Longitude']].shape

accident_data_whole[accident_data_whole['Light_Conditions'] == accident_data_whole['Weather_Conditions']].shape

accident_data_whole[accident_data_whole['Weather_Conditions'] == 9][['Latitude', 'Longitude']].shape